## 3. Tasks and Pipelines
### 3.a List possible pipelines

The Tool Configuration ID allows us to specify the pipeline we want to use and on which Bourreaux we want to run it. By browsing all such pipelines, we can note the Tool Configuration ID for a tool-Bourreaux combination.

In [ ]:
try:

    user_id = str(session_info['user_id'])
    cbrain_api_token = session_info['cbrain_api_token']

except NameError:
    raise UserWarning('Login first.')
 
tools = []

request = {
    'cbrain_api_token': cbrain_api_token,
    'page': 1,
    'per_page': 1000
}

while True:

    response = requests.get(
        url = '/'.join([base_url, 'tool_configs']),
        data = request,
        headers = {'Accept': 'application/json'}
    )

    if response.status_code != requests.codes.ok:
        print('Tools request failed.')
        print(response)
        break

    # Collect the responses on this page then increment
    tools += response.json()
    request['page'] += 1
    
    # Stop requesting responses when we're at the last page
    if len(response.json()) < request['per_page']:
        break 
    
print("{} tool configs found\n".format(str(len(tools))))

As we did before for the files, we can now filter the tools by words in their description. In the following cell we filter to find all tools with the word 'FSL' in the description. The tool config ID is the 'id' field. You can use the 'bourreau_id' and 'tool_id' to verify this is the correct tool you wish to use.

In [ ]:
result = list(filter(lambda f: 'FSL' in f['description'], tools))
print(json.dumps(result, indent=4, sort_keys=True))

### 3.b Submit a task

For this example we will submit a task using a .nii.gz file to FSLFirst (tool_id: 51) on Converter-2 (bourreau_id: 39). This task has a Tool Config ID of 721, which we will use to post a task to CBRAIN.

**Note that this needs a valid userfile ID to run. A public data provider to hold example files for this notebook is in the works.**

In [ ]:
#Store the ID of the file we wish to run a task on. In this case, the first filtered result from the
#files loaded in the previous section. Feel free to change it to a brainscan you have access to.
fileID = result[0]['id']

#Write the post API call
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}
params = (
    ('cbrain_api_token', cbrain_api_token),
)

#Specify the parameters of the task in a dictionary. We are running FSLFirst to register the brain scan.
data = {
  "cbrain_task": {
    'type': 'CbrainTask::FslFirst', 
    'user_id': user_id,
    'tool_config_id': 721, #This specifies the Tool we are running, and the server we are running it on.
    'params': { #These parameters vary from tool to tool.
      'interface_userfile_ids': [fileID], 
      'input_file': fileID, 
      'prefix': 'output', 
      'brain_extracted': False, 
      'three_stage': False, 
      'verbose': False       
    }, 
    'run_number': None, 
    'results_data_provider_id': 179, 
    'cluster_workdir_size': None, 
    'workdir_archived': True, 
    'description': ''}
}

#Convert our parameter dictionary to a json and post the task.
y = json.dumps(data)
response = requests.post('https://portal.cbrain.mcgill.ca/tasks', headers=headers, params=params, data=y)

if response.status_code == 200:
    print(response.text)
    json_format = json.loads(response.text) #save the response to a json file for future use
else:
    print("Task posting failed.")
    print(response.text)

### 3.c Get task information

It is assumed you ran a Task in the previous section before running this section of the code. This allows you to see the output file ID to run further tasks on it, or simply download its contents. It also allows you to track the completion status of the task.

In [ ]:
taskID = json_format[0]["id"]
headers = {
    'Accept': 'application/json',
}
params = (
    ('id', taskID),
    ('cbrain_api_token', cbrain_api_token)
)
url = 'https://portal.cbrain.mcgill.ca/tasks/' + str(taskID)
response = requests.get(url, headers=headers, params=params)
if response.status_code == 200:
    print(response.text)
else:
    print("Task Info retrieval failed.")